In [242]:
import os
import json
import pandas as pd

In [5]:
import os
import json
import pandas as pd

folderPathPorData = '..\\..\\Bases\\PorData\\'
folderPathInePt = '..\\..\\Bases\\InePt\\'

# Função para ler o arquivo JSON e retornar seu conteúdo como uma lista de dicionários
def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: O arquivo {file_path} não foi encontrado.")
    except json.JSONDecodeError:
        print(f"Error: Falha ao decodificar JSON no arquivo {file_path}.")
    return []

# Função para ler arquivos CSV a partir de uma pasta
def read_csv_files_from_folders(folder_path):
    dataframes = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f'Processando arquivo: {file_path}')
                try:
                    df = pd.read_csv(file_path, sep=',', encoding='utf-8', low_memory=False)
                    dataframes.append({
                        'df_folder': os.path.basename(root),
                        'df_name': os.path.splitext(file)[0],
                        'df': df
                    })
                except Exception as e:
                    print(f"Erro ao ler {file}: {e}")
    return dataframes

# Função para tratar o DataFrame de cultura e cinemas
def trata_df_cultura(df_cultura_cinemas):
    df_cultura_cinemas = df_cultura_cinemas[df_cultura_cinemas['03. Âmbito Geográfico'].notnull()]
    last_year = df_cultura_cinemas['01. Ano'].max()
    df_cultura_cinemas = df_cultura_cinemas[df_cultura_cinemas['01. Ano'] == last_year]

    df_cultura_cinemas = df_cultura_cinemas[['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '09. Valor']]
    df_cultura_cinemas = df_cultura_cinemas.rename(columns={
        '01. Ano': 'Ano',
        '02. Nome Região (Portugal)': 'Região',
        '03. Âmbito Geográfico': 'Âmbito Geográfico',
        '09. Valor': 'Valor'
    })
    df_cultura_cinemas['Ano'] = df_cultura_cinemas['Ano'].astype(int)
    df_cultura_cinemas['Valor'] = df_cultura_cinemas['Valor'].astype(int)
    return df_cultura_cinemas.reset_index(drop=True)

# Função para encontrar um DataFrame pelo nome
def find_df_by_name(dataframes, df_name):
    for df_info in dataframes:
        if df_info['df_name'] == df_name:
            return df_info['df']
    raise ValueError(f"DataFrame com o nome '{df_name}' não encontrado.")

# Função para listar cidades com cinemas
def lista_cidades_cinemas(df_cultura_cinemas):
    df_cultura_cinemas = df_cultura_cinemas.groupby(['Região', 'Âmbito Geográfico']).agg({'Valor': 'sum'}).reset_index()
    df_cultura_cinemas = df_cultura_cinemas.sort_values(by='Valor', ascending=False).reset_index(drop=True)
    print("Cidades com mais cinemas:")
    for _, row in df_cultura_cinemas.iterrows():
        print(f"{row['Região']}: {row['Valor']} cinemas")

# Função para tratar o DataFrame de população residente
def trata_df_populacao_residente(df_populaccao_residente):
    df_populaccao_residente = df_populaccao_residente[df_populaccao_residente['04. Âmbito Geográfico'].notnull()]
    last_year = df_populaccao_residente['01. Ano'].max()
    df_populaccao_residente = df_populaccao_residente[df_populaccao_residente['01. Ano'] == last_year]

    df_sexo_pivot = df_populaccao_residente.pivot_table(
        index=['01. Ano', '03. Nome Região (Portugal)'],
        columns='05. Filtro 1',
        values='10. Valor',
        aggfunc='sum',
        fill_value=0
    )

    df_faixa_pivot = df_populaccao_residente.pivot_table(
        index=['01. Ano', '03. Nome Região (Portugal)'],
        columns='06. Filtro 2',
        values='10. Valor',
        aggfunc='sum',
        fill_value=0
    )

    df_polucao_final = pd.merge(df_sexo_pivot, df_faixa_pivot, on=['01. Ano', '03. Nome Região (Portugal)'], how='left')
    df_polucao_final.reset_index(inplace=True)
    df_polucao_final = df_polucao_final.rename(columns={
        '01. Ano': 'Ano',
        '03. Nome Região (Portugal)': 'Região',
        'Homens': 'Homens',
        'Mulheres': 'Mulheres',
        '0 - 4 anos': '0 - 4 anos',
        '5 - 9 anos': '5 - 9 anos',
        '10 - 14 anos': '10 - 14 anos',
        '15 - 19 anos': '15 - 19 anos',
        '20 - 24 anos': '20 - 24 anos',
        '25 - 29 anos': '25 - 29 anos',
        '30 - 34 anos': '30 - 34 anos',
        '35 - 39 anos': '35 - 39 anos',
        '40 - 44 anos': '40 - 44 anos',
        '45 - 49 anos': '45 - 49 anos',
        '50 - 54 anos': '50 - 54 anos',
        '55 - 59 anos': '55 - 59 anos',
        '60 - 64 anos': '60 - 64 anos',
        '65 - 69 anos': '65 - 69 anos',
        '70 - 74 anos': '70 - 74 anos',
        '75 - 79 anos': '75 - 79 anos',
        '80 - 84 anos': '80 - 84 anos',
        '85 ou mais anos': '85 ou mais anos',
        'Total_y': 'Total População'
    })

    df_polucao_final = df_polucao_final.astype({
        'Ano': 'int',
        'Região': 'str',
        'Homens': 'int',
        'Mulheres': 'int',
        '0 - 4 anos': 'int',
        '5 - 9 anos': 'int',
        '10 - 14 anos': 'int',
        '15 - 19 anos': 'int',
        '20 - 24 anos': 'int',
        '25 - 29 anos': 'int',
        '30 - 34 anos': 'int',
        '35 - 39 anos': 'int',
        '40 - 44 anos': 'int',
        '45 - 49 anos': 'int',
        '50 - 54 anos': 'int',
        '55 - 59 anos': 'int',
        '60 - 64 anos': 'int',
        '65 - 69 anos': 'int',
        '70 - 74 anos': 'int',
        '75 - 79 anos': 'int',
        '80 - 84 anos': 'int',
        '85 ou mais anos': 'int',
        'Total População': 'int'
    })

    # Calcular faixas etárias e proporções
    df_polucao_final['Total Crianças'] = df_polucao_final['0 - 4 anos'] + df_polucao_final['5 - 9 anos']
    df_polucao_final['Total Adolescentes'] = df_polucao_final['10 - 14 anos'] + df_polucao_final['15 - 19 anos']
    df_polucao_final['Total Jovens Adultos'] = df_polucao_final['20 - 24 anos']
    df_polucao_final['Total Adultos'] = df_polucao_final[['25 - 29 anos', '30 - 34 anos', '35 - 39 anos', '40 - 44 anos', 
                                                       '45 - 49 anos', '50 - 54 anos', '55 - 59 anos']].sum(axis=1)
    df_polucao_final['Total Idosos'] = df_polucao_final[['60 - 64 anos', '65 - 69 anos', '70 - 74 anos', '75 - 79 anos', 
                                                        '80 - 84 anos', '85 ou mais anos']].sum(axis=1)

    # Proporções de cada faixa etária
    df_polucao_final['Proporção Crianças'] = (df_polucao_final['Total Crianças'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Adolescentes'] = (df_polucao_final['Total Adolescentes'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Jovens Adultos'] = (df_polucao_final['Total Jovens Adultos'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Adultos'] = (df_polucao_final['Total Adultos'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Idosos'] = (df_polucao_final['Total Idosos'] / df_polucao_final['Total População']) * 100

    df_polucao_final = df_polucao_final.drop('Total_x', axis=1)

    return df_polucao_final.reset_index(drop=True)

# Função para listar cidades por grupo etário
def cidades_por_grupo(df, categoria='Todos', n=5):
    categorias = {
        'Crianças': 'Proporção Crianças',
        'Adolescentes': 'Proporção Adolescentes',
        'Jovens Adultos': 'Proporção Jovens Adultos',
        'Adultos': 'Proporção Adultos',
        'Idosos': 'Proporção Idosos',
        'Populosas': 'Total População'
    }
    
    if categoria not in categorias and categoria != 'Todos':
        print("Categoria inválida!")
        return
    
    if categoria == 'Todos':
        categorias_list = categorias.keys()
    else:
        categorias_list = [categoria]

    for cat in categorias_list:
        cidade_ordenada = df[['Ano', 'Região', categorias[cat]]].sort_values(by=categorias[cat], ascending=False).head(n)
        print(f"\nCidades com mais {cat} (%) - Top {n}:")
        for _, row in cidade_ordenada.iterrows():
            proporcao = row[categorias[cat]]
            print(f"{row['Região']}: {int(proporcao)}%")

def trata_df_populacao_densidade(df_densidade_populacional):

    df_densidade_populacional = df_densidade_populacional.dropna(axis=1, how='all')

    df_densidade_populacional = df_densidade_populacional[df_densidade_populacional['03. Nome Região (Portugal)'].notnull()]

    last_year = df_densidade_populacional['01. Ano'].max()
    df_densidade_populacional = df_densidade_populacional[df_densidade_populacional['01. Ano'] == last_year]

    df_densidade_populacional = df_densidade_populacional[df_densidade_populacional['03. Nome Região (Portugal)'] != 'Portugal']

    df_densidade_populacional = df_densidade_populacional[['01. Ano', '03. Nome Região (Portugal)', '04. Âmbito Geográfico', '10. Valor']]

    df_densidade_populacional.rename(columns={
        '01. Ano': 'Ano',
        '03. Nome Região (Portugal)': 'Região',
        '04. Âmbito Geográfico': 'Âmbito Geográfico',
        '10. Valor': 'Valor'
    }, inplace=True)

    df_densidade_populacional = df_densidade_populacional.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Valor': 'float'
    })

    return df_densidade_populacional.reset_index(drop=True)

def trata_crimes_catetoria(df_seguranca_crime_catetoria):
    
    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria.dropna(axis=1, how='all')
    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria[df_seguranca_crime_catetoria['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_seguranca_crime_catetoria['01. Ano'].max()
    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria[df_seguranca_crime_catetoria['01. Ano'] == last_year]

    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria[['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '04. Filtro 1', '09. Valor']]

    df_seguranca_crime_catetoria.rename(columns={
        '01. Ano': 'Ano',
        '02. Nome Região (Portugal)': 'Região',
        '03. Âmbito Geográfico': 'Âmbito Geográfico',
        '04. Filtro 1': 'Categoria',
        '09. Valor': 'Valor'
    }, inplace=True)

    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Categoria': 'str',
        'Valor': 'float'
    })

    # filtering the dataframe to show only rows where 'Total' is not null
    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria[df_seguranca_crime_catetoria['Valor'].notnull()]

    df_seg_categoria = df_seguranca_crime_catetoria.pivot_table(
            index=['Ano', 'Região'],
            columns='Categoria',
            values='Valor',
            aggfunc='sum',
            fill_value=0
    )

    df_seg_categoria.reset_index(inplace=True)

    return df_seg_categoria

def trata_crimes_tipo(df_seg_crimes_tipo):
    df_seg_crimes_tipo = df_seg_crimes_tipo.dropna(axis=1, how='all')
    df_seg_crimes_tipo = df_seg_crimes_tipo[df_seg_crimes_tipo['03. Nome Região (Portugal)'] != 'Portugal']
    df_seg_crimes_tipo = df_seg_crimes_tipo[df_seg_crimes_tipo['03. Nome Região (Portugal)'].notna()]

    last_year = df_seg_crimes_tipo['01. Ano'].max()
    df_seg_crimes_tipo = df_seg_crimes_tipo[df_seg_crimes_tipo['01. Ano'] == last_year]

    df_seg_crimes_tipo = df_seg_crimes_tipo.drop( columns= ['09. Símbolo','08. Escala'], axis=1)

    df_seg_crimes_tipo = df_seg_crimes_tipo[['01. Ano', '03. Nome Região (Portugal)', '04. Âmbito Geográfico', '05. Filtro 1', '10. Valor']]

    df_seg_crimes_tipo.rename(columns={
        '01. Ano': 'Ano',
        '03. Nome Região (Portugal)': 'Região',
        '04. Âmbito Geográfico': 'Âmbito Geográfico',
        '05. Filtro 1': 'Tipo de Crime',
        '10. Valor': 'Valor'
    }, inplace=True)

    df_seg_crimes_tipo = df_seg_crimes_tipo.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Tipo de Crime': 'str',
        'Valor': 'float'
    })

    df_seg_crimes_tipo = df_seg_crimes_tipo.pivot_table(
            index=['Ano', 'Região'],
            columns='Tipo de Crime',
            values='Valor',
            aggfunc='sum',
            fill_value=0
    )

    return df_seg_crimes_tipo.reset_index(drop=False)

def trata_hospitais(df_saude_hospitais):
    df_saude_hospitais = df_saude_hospitais.dropna(axis=1, how='all')
    df_saude_hospitais = df_saude_hospitais[df_saude_hospitais['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_saude_hospitais['01. Ano'].max()
    df_saude_hospitais = df_saude_hospitais[df_saude_hospitais['01. Ano'] == last_year]

    df_saude_hospitais.drop(columns=['08. Símbolo'], inplace=True)

    df_saude_hospitais.rename(columns={
            '01. Ano': 'Ano',
            '02. Nome Região (Portugal)': 'Região',
            '03. Âmbito Geográfico': 'Âmbito Geográfico',
            '04. Filtro 1': 'Tipo hospital',
            '09. Valor': 'Valor'
        }, inplace=True)

    df_saude_hospitais = df_saude_hospitais.astype({
            'Ano': 'int',
            'Região': 'str',
            'Âmbito Geográfico': 'str',
            'Tipo hospital': 'str',
            'Valor': 'float'
        })

    df_saude_hospitais = df_saude_hospitais.pivot_table(
                index=['Ano', 'Região'],
                columns='Tipo hospital',
                values='Valor',
                aggfunc='sum',
                fill_value=0
        )

    df_saude_hospitais.reset_index(inplace=True)

    return df_saude_hospitais

def trata_farmacias(df_saude_farmacias):

    df_saude_farmacias = df_saude_farmacias.dropna(axis=1, how='all')
    df_saude_farmacias = df_saude_farmacias[df_saude_farmacias['02. Nome Região (Portugal)'] != 'Portugal']

    if df_saude_farmacias['08. Símbolo'].nunique() == 1:
        df_saude_farmacias = df_saude_farmacias.drop(columns=['08. Símbolo'])

    last_year = df_saude_farmacias['01. Ano'].max()
    df_saude_farmacias = df_saude_farmacias[df_saude_farmacias['01. Ano'] == last_year]

    df_saude_farmacias.rename(columns={
            '01. Ano': 'Ano',
            '02. Nome Região (Portugal)': 'Região',
            '03. Âmbito Geográfico': 'Âmbito Geográfico',
            '09. Valor': 'Valor'
        }, inplace=True)

    df_saude_farmacias = df_saude_farmacias.astype({
            'Ano': 'int',
            'Região': 'str',
            'Âmbito Geográfico': 'str',
            'Valor': 'float'
        })
    
    return df_saude_farmacias

def trata_ensino_n_superior(df_edu_ensino_n_superior):
    df_edu_ensino_n_superior = df_edu_ensino_n_superior.dropna(axis=1, how='all')
    df_edu_ensino_n_superior = df_edu_ensino_n_superior[df_edu_ensino_n_superior['02. Nome Região (Portugal)'] != 'Portugal']

    df_edu_ensino_n_superior = df_edu_ensino_n_superior.drop(columns=['08. Símbolo'])

    last_year = df_edu_ensino_n_superior['01. Ano'].max()
    df_edu_ensino_n_superior = df_edu_ensino_n_superior[df_edu_ensino_n_superior['01. Ano'] == last_year]

    df_edu_ensino_n_superior.rename(columns={
            '01. Ano': 'Ano',
            '02. Nome Região (Portugal)': 'Região',
            '03. Âmbito Geográfico': 'Âmbito Geográfico',
            '04. Filtro 1': 'Nível de Ensino',
            '05. Filtro 2': 'Tipo de Escola',
            '09. Valor': 'Valor'
        }, inplace=True)

    df_edu_ensino_n_superior = df_edu_ensino_n_superior.astype({
            'Ano': 'int',
            'Região': 'str',
            'Âmbito Geográfico': 'str',
            'Nível de Ensino': 'str',
            'Tipo de Escola': 'str',
            'Valor': 'float'
        })

    df_edu_escola_nivel = df_edu_ensino_n_superior.pivot_table(
            index=['Ano', 'Região'],
            columns='Nível de Ensino',
            values='Valor',
            aggfunc='sum',
            fill_value=0
        )

    df_edu_escola_nivel.reset_index(inplace=True)

    df_edu_escola_tipo= df_edu_ensino_n_superior.pivot_table(
            index=['Ano', 'Região'],
            columns='Tipo de Escola',
            values='Valor',
            aggfunc='sum',
            fill_value=0
        )

    df_edu_escola_tipo.reset_index(inplace=True)

    df_edu_escola_final = pd.merge(df_edu_escola_tipo, df_edu_escola_nivel, on=['Ano', 'Região'], how='left')
    df_edu_escola_final.reset_index(inplace=True)

    df_edu_escola_final.drop(columns=['index','Total_x'], inplace=True)
    df_edu_escola_final.rename(columns={'Total_y': 'Total Estabelecimentos'}, inplace=True)

    return df_edu_escola_final


def trata_ensino_superior(df_edu_ensino_superior):
    
    df_edu_ensino_superior = df_edu_ensino_superior.dropna(axis=1, how='all')
    df_edu_ensino_superior = df_edu_ensino_superior[df_edu_ensino_superior['02. Nome Região (Portugal)'] != 'Portugal']

    df_edu_ensino_superior = df_edu_ensino_superior.drop(columns=['08. Símbolo'])

    last_year = df_edu_ensino_superior['01. Ano'].max()
    df_edu_ensino_superior = df_edu_ensino_superior[df_edu_ensino_superior['01. Ano'] == last_year]

    df_edu_ensino_superior.rename(columns={
            '01. Ano': 'Ano',
            '02. Nome Região (Portugal)': 'Região',
            '03. Âmbito Geográfico': 'Âmbito Geográfico',
            '04. Filtro 1': 'Tipo de Escola',
            '09. Valor': 'Valor'
        }, inplace=True)

    df_edu_ensino_superior = df_edu_ensino_superior.astype({
            'Ano': 'int',
            'Região': 'str',
            'Âmbito Geográfico': 'str',
            'Tipo de Escola': 'str',
            'Valor': 'float'
        })

    df_edu_ensino_superior = df_edu_ensino_superior.pivot_table(
            index=['Ano', 'Região'],
            columns='Tipo de Escola',
            values='Valor',
            aggfunc='sum',
            fill_value=0
        )

    df_edu_ensino_superior.reset_index(inplace=True)

    return df_edu_ensino_superior


def trata_econ_depositos(df_econ_depositos):
    df_econ_depositos = df_econ_depositos.dropna(axis=1, how='all')

    df_econ_depositos = df_econ_depositos[df_econ_depositos['02. Nome Região (Portugal)'] != 'Portugal']

    df_econ_depositos = df_econ_depositos.drop(columns=['08. Símbolo'])

    last_year = df_econ_depositos['01. Ano'].max()
    df_econ_depositos = df_econ_depositos[df_econ_depositos['01. Ano'] == last_year]

    df_econ_depositos.rename(columns={
                '01. Ano': 'Ano',
                '02. Nome Região (Portugal)': 'Região',
                '03. Âmbito Geográfico': 'Âmbito Geográfico',
                '04. Filtro 1': 'Deposito',
                '09. Valor': 'Valor'
            }, inplace=True)

    df_econ_depositos = df_econ_depositos.astype({
                'Ano': 'int',
                'Região': 'str',
                'Âmbito Geográfico': 'str',
                'Deposito': 'str',
                'Valor': 'float'
            })

    df_econ_depositos = df_econ_depositos.pivot_table(
                index=['Ano', 'Região'],
                columns='Deposito',
                values='Valor',
                aggfunc='sum',
                fill_value=0
            )

    df_econ_depositos.reset_index(inplace=True)

    return df_econ_depositos


def trata_econ_bancos(df_econ_estabelecimentos_bancos):
    
    df_econ_estabelecimentos_bancos = df_econ_estabelecimentos_bancos.dropna(axis=1, how='all')

    df_econ_estabelecimentos_bancos = df_econ_estabelecimentos_bancos[df_econ_estabelecimentos_bancos['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_econ_estabelecimentos_bancos['01. Ano'].max()
    df_econ_estabelecimentos_bancos = df_econ_estabelecimentos_bancos[df_econ_estabelecimentos_bancos['01. Ano'] == last_year]

    df_econ_estabelecimentos_bancos.rename(columns={
                    '01. Ano': 'Ano',
                    '02. Nome Região (Portugal)': 'Região',
                    '03. Âmbito Geográfico': 'Âmbito Geográfico',
                    '09. Valor': 'Valor'
                }, inplace=True)

    df_econ_estabelecimentos_bancos = df_econ_estabelecimentos_bancos.astype({
                    'Ano': 'int',
                    'Região': 'str',
                    'Âmbito Geográfico': 'str',
                    'Valor': 'float'
                })

    return df_econ_estabelecimentos_bancos


def trata_df_desemprego(df_desemprego):
    df_desemprego = df_desemprego.dropna(axis=1, how='all')
    df_desemprego = df_desemprego[df_desemprego['02. Nome Região (Portugal)'] != 'Portugal']
    df_desemprego = df_desemprego.drop(columns=['08. Símbolo'])

    last_year = df_desemprego['01. Ano'].max()
    df_desemprego = df_desemprego[df_desemprego['01. Ano'] == last_year]

    df_desemprego.rename(columns={
                '01. Ano': 'Ano',
                '02. Nome Região (Portugal)': 'Região',
                '03. Âmbito Geográfico': 'Âmbito Geográfico',
                '04. Filtro 1': 'Faixa Etária',
                '09. Valor': 'Valor'
            }, inplace=True)

    df_desemprego = df_desemprego.astype({
                'Ano': 'int',
                'Região': 'str',
                'Âmbito Geográfico': 'str',
                'Faixa Etária': 'str',
                'Valor': 'float'
            })
    df_desemprego = df_desemprego.reset_index(drop=True)

    df_desemprego = df_desemprego.pivot_table(
                    index=['Ano', 'Região'],
                    columns='Faixa Etária',
                    values='Valor',
                    aggfunc='sum',
                    fill_value=0
                )

    df_desemprego.reset_index(inplace=True)

    return df_desemprego

def trata_populacao_empregada_escolaridade(df_polacao_empregada):
    df_polacao_empregada = df_polacao_empregada.dropna(axis=1, how='all')
    df_polacao_empregada = df_polacao_empregada[df_polacao_empregada['02. Nome Região (Portugal)'] != 'Portugal']
    df_polacao_empregada = df_polacao_empregada.drop(columns=['08. Símbolo'])

    last_year = df_polacao_empregada['01. Ano'].max()
    df_polacao_empregada = df_polacao_empregada[df_polacao_empregada['01. Ano'] == last_year]

    df_polacao_empregada.rename(columns={
                    '01. Ano': 'Ano',
                    '02. Nome Região (Portugal)': 'Região',
                    '03. Âmbito Geográfico': 'Âmbito Geográfico',
                    '04. Filtro 1': 'Escolaridade',
                    '09. Valor': 'Valor'
                }, inplace=True)

    df_polacao_empregada = df_polacao_empregada.astype({
                    'Ano': 'int',
                    'Região': 'str',
                    'Âmbito Geográfico': 'str',
                    'Escolaridade': 'str',
                    'Valor': 'float'
                })

    df_polacao_empregada = df_polacao_empregada.pivot_table(
                        index=['Ano', 'Região'],
                        columns='Escolaridade',
                        values='Valor',
                        aggfunc='sum',
                        fill_value=0
                    )

    df_polacao_empregada.reset_index(inplace=True)

    return df_polacao_empregada

def trata_econ_bancos(df_econ_estabelecimentos_bancos):
    df_econ_estabelecimentos_bancos = df_econ_estabelecimentos_bancos.dropna(axis=1, how='all')

    df_econ_estabelecimentos_bancos = df_econ_estabelecimentos_bancos[df_econ_estabelecimentos_bancos['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_econ_estabelecimentos_bancos['01. Ano'].max()
    df_econ_estabelecimentos_bancos = df_econ_estabelecimentos_bancos[df_econ_estabelecimentos_bancos['01. Ano'] == last_year]

    df_econ_estabelecimentos_bancos.rename(columns={
                    '01. Ano': 'Ano',
                    '02. Nome Região (Portugal)': 'Região',
                    '03. Âmbito Geográfico': 'Âmbito Geográfico',
                    '09. Valor': 'Valor'
                }, inplace=True)

    df_econ_estabelecimentos_bancos = df_econ_estabelecimentos_bancos.astype({
                    'Ano': 'int',
                    'Região': 'str',
                    'Âmbito Geográfico': 'str',
                    'Valor': 'float'
                })

    return df_econ_estabelecimentos_bancos

def trata_populacao_empregada_ramo(df_polacao_empregada_ramo):
    df_polacao_empregada_ramo = df_polacao_empregada_ramo.dropna(axis=1, how='all')
    df_polacao_empregada_ramo = df_polacao_empregada_ramo[df_polacao_empregada_ramo['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_polacao_empregada_ramo['01. Ano'].max()
    df_polacao_empregada_ramo = df_polacao_empregada_ramo[df_polacao_empregada_ramo['01. Ano'] == last_year]

    df_polacao_empregada_ramo = df_polacao_empregada_ramo.drop(columns=['08. Símbolo'])

    df_polacao_empregada_ramo.rename(columns={
                        '01. Ano': 'Ano',
                        '02. Nome Região (Portugal)': 'Região',
                        '03. Âmbito Geográfico': 'Âmbito Geográfico',
                        '04. Filtro 1': 'Ramo de Atividade',
                        '09. Valor': 'Valor'
                    }, inplace=True)

    df_polacao_empregada_ramo = df_polacao_empregada_ramo.astype({
                        'Ano': 'int',
                        'Região': 'str',
                        'Âmbito Geográfico': 'str',
                        'Ramo de Atividade': 'str',
                        'Valor': 'float'
                    })

    df_polacao_empregada_ramo = df_polacao_empregada_ramo.pivot_table(
                            index=['Ano', 'Região'],
                            columns='Ramo de Atividade',
                            values='Valor',
                            aggfunc='sum',
                            fill_value=0
                        )
    df_polacao_empregada_ramo.reset_index(inplace=True)

    return df_polacao_empregada_ramo

def trata_populacao_estrangeira(df_populacao_estrangeira):
    df_populacao_estrangeira = df_populacao_estrangeira.dropna(axis=1, how='all')
    df_populacao_estrangeira = df_populacao_estrangeira[df_populacao_estrangeira['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_populacao_estrangeira['01. Ano'].max()
    df_populacao_estrangeira = df_populacao_estrangeira[df_populacao_estrangeira['01. Ano'] == last_year]

    df_populacao_estrangeira = df_populacao_estrangeira.drop(columns=['08. Símbolo'])

    df_populacao_estrangeira.rename(columns={
                            '01. Ano': 'Ano',
                            '02. Nome Região (Portugal)': 'Região',
                            '03. Âmbito Geográfico': 'Âmbito Geográfico',
                            '04. Filtro 1': 'Nacionalidade',
                            '09. Valor': 'Valor'
                        }, inplace=True)

    df_populacao_estrangeira = df_populacao_estrangeira.astype({
                            'Ano': 'int',
                            'Região': 'str',
                            'Âmbito Geográfico': 'str',
                            'Nacionalidade': 'str',
                            'Valor': 'float'
                        })

    df_populacao_estrangeira = df_populacao_estrangeira.pivot_table(
                                index=['Ano', 'Região'],
                                columns='Nacionalidade',
                                values='Valor',
                                aggfunc='sum',
                                fill_value=0
                            )
    df_populacao_estrangeira.reset_index(inplace=True)

    return df_populacao_estrangeira

def trata_moradia_edificios(df_moradia_edificios):
    df_moradia_edificios = df_moradia_edificios.dropna(axis=1, how='all')
    df_moradia_edificios = df_moradia_edificios[df_moradia_edificios['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_moradia_edificios['01. Ano'].max()
    df_moradia_edificios = df_moradia_edificios[df_moradia_edificios['01. Ano'] == last_year]

    df_moradia_edificios.rename(columns={
                                '01. Ano': 'Ano',
                                '02. Nome Região (Portugal)': 'Região',
                                '03. Âmbito Geográfico': 'Âmbito Geográfico',
                                '04. Filtro 1': 'Tipo de Obra',
                                '05. Filtro 2': 'Fim que se destina',
                                '09. Valor': 'Valor'
                            }, inplace=True)

    df_moradia_edificios = df_moradia_edificios.astype({
                                'Ano': 'int',
                                'Região': 'str',
                                'Âmbito Geográfico': 'str',
                                'Tipo de Obra': 'str',
                                'Fim que se destina': 'str',
                                'Valor': 'float'
                            })

    df_moradia_edificios_tipo = df_moradia_edificios.pivot_table(
                                    index=['Ano', 'Região'],
                                    columns='Tipo de Obra',
                                    values='Valor',
                                    aggfunc='sum',
                                    fill_value=0
                                )
    df_moradia_edificios_tipo.reset_index(inplace=True)

    df_moradia_edificios_destino = df_moradia_edificios.pivot_table(
                                    index=['Ano', 'Região'],
                                    columns='Fim que se destina',
                                    values='Valor',
                                    aggfunc='sum',
                                    fill_value=0
                                )
    df_moradia_edificios_destino.reset_index(inplace=True)

    df_moradia_edificios_final = pd.merge(df_moradia_edificios_destino, df_moradia_edificios_tipo, on=['Ano', 'Região'], how='left')
    df_moradia_edificios_final.reset_index(inplace=True)

    df_moradia_edificios_final.drop(columns=['index','Total_x'], inplace=True)

    return df_moradia_edificios_final


def trata_construcoes_novas(df_construcoes_novas):
    df_construcoes_novas = df_construcoes_novas.dropna(axis=1, how='all')
    df_construcoes_novas = df_construcoes_novas[df_construcoes_novas['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_construcoes_novas['01. Ano'].max()
    df_construcoes_novas = df_construcoes_novas[df_construcoes_novas['01. Ano'] == last_year]

    df_construcoes_novas.rename(columns={
                                    '01. Ano': 'Ano',
                                    '02. Nome Região (Portugal)': 'Região',
                                    '03. Âmbito Geográfico': 'Âmbito Geográfico',
                                    '04. Filtro 1': 'Qtd. Quartos',
                                    '09. Valor': 'Valor'
                                }, inplace=True)

    df_construcoes_novas = df_construcoes_novas.astype({
                                    'Ano': 'int',
                                    'Região': 'str',
                                    'Âmbito Geográfico': 'str',
                                    'Qtd. Quartos': 'str',
                                    'Valor': 'float'
                                })

    df_construcoes_novas = df_construcoes_novas.pivot_table(
                                index=['Ano', 'Região'],
                                columns='Qtd. Quartos',
                                values='Valor',
                                aggfunc='sum',
                                fill_value=0
                            )
    df_construcoes_novas.reset_index(inplace=True)

    return df_construcoes_novas

def trata_sal_ganho_medio_mensal(df_sal_ganho_medio_mensal):
    df_sal_ganho_medio_mensal = df_sal_ganho_medio_mensal.dropna(axis=1, how='all')
    df_sal_ganho_medio_mensal = df_sal_ganho_medio_mensal[df_sal_ganho_medio_mensal['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_sal_ganho_medio_mensal['01. Ano'].max()
    df_sal_ganho_medio_mensal = df_sal_ganho_medio_mensal[df_sal_ganho_medio_mensal['01. Ano'] == last_year]

    df_sal_ganho_medio_mensal.drop(columns=['08. Símbolo'], inplace=True)

    df_sal_ganho_medio_mensal.rename(columns={
        '01. Ano': 'Ano',
        '02. Nome Região (Portugal)': 'Região',
        '03. Âmbito Geográfico': 'Âmbito Geográfico',
        '09. Valor': 'Valor'
    }, inplace=True)

    df_sal_ganho_medio_mensal = df_sal_ganho_medio_mensal.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Valor': 'float'
    })

    return df_sal_ganho_medio_mensal

def trata_sal_ganho_medio_mensal_escol(df_sal_ganho_medio_mensal_escol):
    df_sal_ganho_medio_mensal_escol = df_sal_ganho_medio_mensal_escol.dropna(axis=1, how='all')
    df_sal_ganho_medio_mensal_escol = df_sal_ganho_medio_mensal_escol[df_sal_ganho_medio_mensal_escol['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_sal_ganho_medio_mensal_escol['01. Ano'].max()
    df_sal_ganho_medio_mensal_escol = df_sal_ganho_medio_mensal_escol[df_sal_ganho_medio_mensal_escol['01. Ano'] == last_year]

    df_sal_ganho_medio_mensal_escol.drop(columns=['08. Símbolo'], inplace=True)

    df_sal_ganho_medio_mensal_escol.rename(columns={
        '01. Ano': 'Ano',
        '02. Nome Região (Portugal)': 'Região',
        '03. Âmbito Geográfico': 'Âmbito Geográfico',
        '04. Filtro 1': 'Nível de Escolaridade',
        '09. Valor': 'Valor'
    }, inplace=True)

    df_sal_ganho_medio_mensal_escol = df_sal_ganho_medio_mensal_escol.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Nível de Escolaridade': 'str',
        'Valor': 'float'
    })

    df_sal_ganho_medio_mensal_escol = df_sal_ganho_medio_mensal_escol.pivot_table(
                                    index=['Ano', 'Região'],
                                    columns='Nível de Escolaridade',
                                    values='Valor',
                                    aggfunc='sum',
                                    fill_value=0
                                )
    df_sal_ganho_medio_mensal_escol.reset_index(inplace=True)

    return df_sal_ganho_medio_mensal_escol

def trata_econ_poder_compra():

    file_path = '..\\..\\Bases\\PorData\\economia\\Municipios_Proporcao_de_poder_de_compra.xlsx'

    excel_data = pd.ExcelFile(file_path)

    # Carregar a planilha 'Quadro'
    df = pd.read_excel(excel_data, sheet_name='Quadro')

    # Limpar e extrair as colunas e dados relevantes
    df_econ_poder_compra = df.iloc[10:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
    df_econ_poder_compra.columns = ['Âmbito Geográfico', 'Anos', '1993', '2000', '2002', '2007', '2009', '2011', '2013', '2015', '2017', '2019', '2021']

    # Remover linhas completamente vazias
    df_econ_poder_compra = df_econ_poder_compra.dropna(how='all')

    # Resetar o índice
    df_econ_poder_compra.reset_index(drop=True, inplace=True)

    df_econ_poder_compra = df_econ_poder_compra.drop(0, axis=0).reset_index(drop=True)

    df_econ_poder_compra = df_econ_poder_compra[df_econ_poder_compra['Âmbito Geográfico'] == 'Município']

    df_econ_poder_compra = df_econ_poder_compra[['Âmbito Geográfico', 'Anos', df_econ_poder_compra.columns[-1]]]

    ultima_coluna = df_econ_poder_compra.columns[-1]
    df_econ_poder_compra.rename(columns={
        'Âmbito Geográfico': 'Âmbito Geográfico',
        'Anos': 'Região',
        ultima_coluna: 'Valor'
    }, inplace=True)

    df_econ_poder_compra = df_econ_poder_compra.astype({
        'Âmbito Geográfico': 'str',
        'Região': 'str',
        'Valor': 'float'
    })

    return df_econ_poder_compra

In [97]:
def trata_seg_taxa_crime():
    file_path = '..\\..\\Bases\\InePt\\seguranca\\INE_PT-Taxa-de-criminalidade.xls'

    excel_data = pd.ExcelFile(file_path)

    # Carregar a planilha 'Quadro'
    df_seg_tax_crime = pd.read_excel(excel_data, sheet_name='Quadro')

    df_seg_tax_crime= df_seg_tax_crime.iloc[10:, [0, 1, 2, 4, 6, 8, 10, 12, 14]]
    df_seg_tax_crime.columns = [
        'Regiao', 
        'Codigo', 
        'Total', 
        'Crimes contra a integridade física', 
        'Furto/roubo por esticão e na via pública', 
        'Furto de veículo e em veículo motorizado', 
        'Condução de veículo com taxa de álcool igual ou superior a 1,2g/l',
        'Condução sem habilitação legal', 
        'Crimes contra o património'
        ]

    # Remover linhas completamente vazias
    df_seg_tax_crime = df_seg_tax_crime.dropna(how='all')

    df_seg_tax_crime['Codigo'] = df_seg_tax_crime['Codigo'].astype(str)
    df_seg_tax_crime = df_seg_tax_crime[df_seg_tax_crime['Codigo'].str.len() == 7]

    df_seg_tax_crime = df_seg_tax_crime.astype({
        'Regiao':'str',
        'Codigo':'str',
        'Total': 'float',
        'Crimes contra a integridade física': 'float',
        'Furto/roubo por esticão e na via pública': 'float',
        'Furto de veículo e em veículo motorizado': 'float',
        'Condução de veículo com taxa de álcool igual ou superior a 1,2g/l': 'float',
        'Condução sem habilitação legal': 'float',
        'Crimes contra o património': 'float'
    })

    return df_seg_tax_crime

def trata_mora_valor_renda():
        file_path = '..\\..\\Bases\\InePt\\moradia\\INE_PT-Valor-mediano-das-rendas.xls'

        excel_data = pd.ExcelFile(file_path)

        # Carregar a planilha 'Quadro'
        df_mora_valor_rendas = pd.read_excel(excel_data, sheet_name='Quadro')

        df_mora_valor_rendas= df_mora_valor_rendas.iloc[9:, [0, 1, 2, 3]]
        df_mora_valor_rendas.columns = [
                'Regiao', 
                'Codigo', 
                'Valor mediano das rendas por m2',
                'lixo'
        ]

        # Remover linhas completamente vazias
        df_mora_valor_rendas = df_mora_valor_rendas.dropna(how='all')

        df_mora_valor_rendas.drop(columns=['lixo'], inplace=True)

        df_mora_valor_rendas['Codigo'] = df_mora_valor_rendas['Codigo'].astype(str)
        # df_1= df_mora_valor_rendas[df_mora_valor_rendas['Codigo'].str.match(r'^\d{2}$')]
        # df_3= df_mora_valor_rendas[df_mora_valor_rendas['Codigo'].str.match(r'^\d{3}$')]

        df_mora_valor_rendas = df_mora_valor_rendas[df_mora_valor_rendas['Codigo'].str.len() == 7]

        # Criar nova coluna com valor estimado para apartamento de 100m²
        df_mora_valor_rendas['Valor apartamento 100m²'] = df_mora_valor_rendas['Valor mediano das rendas por m2'] * 100

         # Criar nova coluna com valor estimado para apartamento de 80m²
        df_mora_valor_rendas['Valor apartamento 80m²'] = df_mora_valor_rendas['Valor mediano das rendas por m2'] * 80


        return df_mora_valor_rendas


In [ ]:
file_path = 'pordataFiles.json'
arquivos = read_json_file(file_path)

In [ ]:
dataframes = read_csv_files_from_folders(folderPathPorData)

In [96]:
df_taxa_crimes = trata_seg_taxa_crime()
df_taxa_crimes

,Regiao,Codigo,Total,Crimes contra a integridade física,Furto/roubo por esticão e na via pública,Furto de veículo e em veículo motorizado,"Condução de veículo com taxa de álcool igual ou superior a 1,2g/l",Condução sem habilitação legal,Crimes contra o património
14,Arcos de Valdevez,1111601,30.9,5.0,NaN,1.4,2.0,1.1,9.3
15,Caminha,1111602,35.6,6.9,NaN,2.5,2.8,1.3,15.5
16,Melgaço,1111603,41.9,5.7,NaN,1.6,2.2,1.6,18.9
17,Monção,1111604,18.3,2.5,0.0,1.2,2.2,1.6,7.9
18,Paredes de Coura,1111605,27.7,3.0,0.0,1.0,4.0,0.9,12.9
...,...,...,...,...,...,...,...,...,...
352,Ribeira Brava,3003107,19.5,5.9,NaN,0.6,1.3,NaN,5.1
353,Santa Cruz,3003108,17.3,4.2,0.3,0.5,1.7,0.5,6.8
354,Santana,3003109,16.4,4.1,0.0,0.9,1.2,0.8,6.8
355,São Vicente,3003110,17.2,3.5,0.0,0.6,2.9,NaN,6.1


In [92]:
df_mora_valor_rendas = trata_mora_valor_renda()
df_mora_valor_rendas

,Regiao,Codigo,Valor mediano das rendas por m2,Valor apartamento 100m²,Valor apartamento 80m²
12,Arcos de Valdevez,1111601,4.64,464.0,371.2
13,Caminha,1111602,5.83,583.0,466.4
14,Melgaço,1111603,3.29,329.0,263.2
15,Monção,1111604,6.58,658.0,526.4
16,Paredes de Coura,1111605,3.84,384.0,307.2
...,...,...,...,...,...
641,Ribeira Brava,3003107,NaN,NaN,NaN
642,Santa Cruz,3003108,9.02,902.0,721.6
643,Santana,3003109,NaN,NaN,NaN
644,São Vicente,3003110,NaN,NaN,NaN


In [93]:
df_mora_valor_rendas[df_mora_valor_rendas['Regiao'].str.startswith('Set')]

,Regiao,Codigo,Valor mediano das rendas por m2,Valor apartamento 100m²,Valor apartamento 80m²
534,Setúbal,1B01512,9.65,965.0,772.0


In [246]:
df_cultura_cinemas = find_df_by_name(dataframes, '623-recintos-de-cinema')
df_cultura_cinemas = trata_df_cultura(df_cultura_cinemas)
lista_cidades_cinemas(df_cultura_cinemas)

Cidades com mais cinemas:
Lisboa: 16 cinemas
Porto: 7 cinemas
Coimbra: 5 cinemas
Vila Nova de Gaia: 4 cinemas
Braga: 4 cinemas
Leiria: 4 cinemas
Viseu: 3 cinemas
Barreiro: 3 cinemas
Maia: 3 cinemas
Cascais: 3 cinemas
Oeiras: 3 cinemas
Almada: 3 cinemas
Guimarães: 3 cinemas
Funchal: 3 cinemas
Setúbal: 2 cinemas
Viana do Castelo: 2 cinemas
Matosinhos: 2 cinemas
Estarreja: 2 cinemas
Santarém: 2 cinemas
Vila Franca de Xira: 2 cinemas
Faro: 2 cinemas
São João da Madeira: 2 cinemas
Seixal: 2 cinemas
Caldas da Rainha: 2 cinemas
Castelo Branco: 2 cinemas
Loulé: 2 cinemas
Torres Novas: 2 cinemas
Torres Vedras: 2 cinemas
Guarda: 2 cinemas
Aveiro: 2 cinemas
Figueira da Foz: 2 cinemas
Espinho: 2 cinemas
Odivelas: 2 cinemas
Ponta Delgada: 2 cinemas
Angra do Heroísmo: 2 cinemas
Montijo: 2 cinemas
Portimão: 2 cinemas
Évora: 2 cinemas
Vila Real: 2 cinemas
Horta: 1 cinemas
Gondomar: 1 cinemas
Oliveira do Bairro: 1 cinemas
Portel: 1 cinemas
Portalegre: 1 cinemas
Grândola: 1 cinemas
Sesimbra: 1 cinemas
S

In [247]:
df_cultura_espetaculos = find_df_by_name(dataframes, '631-sessoes-de-espetaculos-ao-vivo')
df_cultura_espetaculos = trata_df_cultura(df_cultura_espetaculos)
lista_cidades_cinemas(df_cultura_espetaculos)

Cidades com mais cinemas:
Lisboa: 7906 cinemas
Porto: 4929 cinemas
Coimbra: 1728 cinemas
Leiria: 790 cinemas
Oeiras: 784 cinemas
Funchal: 738 cinemas
Almada: 637 cinemas
Vila Nova de Gaia: 608 cinemas
Cascais: 582 cinemas
Sintra: 577 cinemas
Faro: 546 cinemas
Évora: 525 cinemas
Aveiro: 500 cinemas
Braga: 491 cinemas
Viseu: 488 cinemas
Loulé: 422 cinemas
Beja: 400 cinemas
Setúbal: 398 cinemas
Matosinhos: 383 cinemas
Guimarães: 355 cinemas
Santa Maria da Feira: 355 cinemas
Alcobaça: 343 cinemas
Ílhavo: 338 cinemas
Vila Nova de Famalicão: 331 cinemas
Torres Vedras: 307 cinemas
Palmela: 295 cinemas
Guarda: 295 cinemas
Montijo: 291 cinemas
Vila Franca de Xira: 279 cinemas
Angra do Heroísmo: 260 cinemas
Ourém: 256 cinemas
São João da Madeira: 255 cinemas
Barcelos: 252 cinemas
Odivelas: 250 cinemas
Ponte de Lima: 236 cinemas
Vila Real: 235 cinemas
Aljustrel: 229 cinemas
Lagos: 225 cinemas
Marinha Grande: 220 cinemas
Loures: 216 cinemas
Sesimbra: 216 cinemas
Amadora: 208 cinemas
Monção: 203 ci

In [248]:
df_populaccao_residente = find_df_by_name(dataframes, '1-populacao-residente-por-sexo-e-grupo-etario')
df_populaccao_final = trata_df_populacao_residente(df_populaccao_residente)
cidades_por_grupo(df_populaccao_final, categoria='Todos', n = 10) # Todos Crianças Adolescentes Jovens Adultos Adultos Idosos Populosas


Cidades com mais Crianças (%) - Top 10:
Ribeira Grande: 11%
Montijo: 10%
Odivelas: 10%
Loures: 10%
Lagoa [R.A.A.]: 10%
Amadora: 10%
Portimão: 9%
Sintra: 9%
Moura: 9%
Moita: 9%

Cidades com mais Adolescentes (%) - Top 10:
Ribeira Grande: 13%
Alcochete: 12%
Arruda dos Vinhos: 12%
Lagoa [R.A.A.]: 12%
Mafra: 12%
Vila Franca do Campo: 11%
Campo Maior: 11%
Benavente: 11%
Câmara de Lobos: 11%
Nordeste: 11%

Cidades com mais Jovens Adultos (%) - Top 10:
Câmara de Lobos: 7%
Ribeira Grande: 7%
Lagoa [R.A.A.]: 6%
Povoação: 6%
Vila do Porto: 6%
Ribeira Brava: 6%
Castelo de Paiva: 6%
Marco de Canaveses: 6%
Lousada: 6%
Paços de Ferreira: 6%

Cidades com mais Adultos (%) - Top 10:
Corvo: 53%
Odemira: 52%
Santa Cruz: 51%
Ponta Delgada: 51%
Lagoa [R.A.A.]: 51%
Santa Cruz das Flores: 50%
Ribeira Grande: 50%
Lousada: 50%
Vila Franca do Campo: 50%
Paços de Ferreira: 50%

Cidades com mais Idosos (%) - Top 10:
Vinhais: 54%
Oleiros: 53%
Alcoutim: 53%
Pampilhosa da Serra: 52%
Almeida: 52%
Sabugal: 52%
Vimios

In [249]:
df_densidade_populacional = find_df_by_name(dataframes, '4-densidade-populacional')
df_densidade_populacional = trata_df_populacao_densidade(df_densidade_populacional)

In [250]:
df_densidade_populacional = df_densidade_populacional.sort_values(by=['Valor'], ascending=False).reset_index(drop=True)
print("Densidade populacional por região:")

Densidade populacional por região:


In [269]:
df_seguranca_crime_catetoria = find_df_by_name(dataframes, '560-crimes-por-categoria')
df_seg_categoria = trata_crimes_catetoria(df_seguranca_crime_catetoria)
df_seg_categoria

Categoria,Ano,Região,1. Contra as pessoas,2. Contra o património,3. Contra a vida em sociedade,4. Contra o Estado,"5. Contra a identidade cultural, integridade pessoal",6. Contra animais companhia,7. Legislação Avulsa e Outros,Total
0,2024,Abrantes,217.0,473.0,106.0,7.0,0.0,6.0,31.0,840.0
1,2024,Aguiar da Beira,44.0,53.0,31.0,0.0,0.0,0.0,10.0,140.0
2,2024,Alandroal,34.0,37.0,11.0,4.0,0.0,0.0,5.0,91.0
3,2024,Albergaria-a-Velha,248.0,364.0,152.0,20.0,0.0,0.0,45.0,830.0
4,2024,Albufeira,900.0,2291.0,325.0,60.0,4.0,13.0,226.0,3819.0
...,...,...,...,...,...,...,...,...,...,...
303,2024,Vouzela,100.0,99.0,24.0,4.0,0.0,0.0,5.0,234.0
304,2024,Águeda,343.0,581.0,335.0,68.0,0.0,7.0,86.0,1420.0
305,2024,Évora,407.0,762.0,121.0,23.0,0.0,10.0,117.0,1441.0
306,2024,Ílhavo,330.0,609.0,316.0,69.0,0.0,6.0,134.0,1466.0


In [270]:
df_seg_crimes_tipo = find_df_by_name(dataframes, '559-crimes-registados-pelas-policias-por-tipo-de-crime')
df_seg_crimes_tipo = trata_crimes_tipo(df_seg_crimes_tipo)
df_seg_crimes_tipo

Tipo de Crime,Ano,Região,Furto em edifício comercial ou industrial,Furto em residência,Furto em veículo motorizado,Total,Violência doméstica contra cônjuge ou análogos
0,2024,Abrantes,26.0,16.0,0.0,840.0,65.0
1,2024,Aguiar da Beira,0.0,6.0,0.0,140.0,16.0
2,2024,Alandroal,0.0,0.0,0.0,91.0,12.0
3,2024,Albergaria-a-Velha,5.0,10.0,0.0,830.0,72.0
4,2024,Albufeira,67.0,114.0,0.0,3819.0,204.0
...,...,...,...,...,...,...,...
303,2024,Vouzela,0.0,11.0,0.0,234.0,18.0
304,2024,Águeda,18.0,23.0,0.0,1420.0,126.0
305,2024,Évora,28.0,39.0,0.0,1441.0,111.0
306,2024,Ílhavo,17.0,44.0,0.0,1466.0,94.0


In [271]:
df_saude_hospitais = find_df_by_name(dataframes, '470-hospitais-por-natureza-institucional')
df_saude_hospitais = trata_hospitais(df_saude_hospitais)
df_saude_hospitais

Tipo hospital,Ano,Região,Parceria público-privada,Privado,Público,Total
0,2023,Abrantes,0.0,0.0,1.0,1.0
1,2023,Aguiar da Beira,0.0,0.0,0.0,0.0
2,2023,Alandroal,0.0,0.0,0.0,0.0
3,2023,Albergaria-a-Velha,0.0,0.0,0.0,0.0
4,2023,Albufeira,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...
303,2023,Vouzela,0.0,0.0,0.0,0.0
304,2023,Águeda,0.0,0.0,1.0,1.0
305,2023,Évora,0.0,1.0,1.0,2.0
306,2023,Ílhavo,0.0,0.0,0.0,0.0


In [254]:
df_saude_farmacias = find_df_by_name(dataframes, '464-farmacias')
df_saude_farmacias = trata_farmacias(df_saude_farmacias)
df_saude_farmacias

,Ano,Região,Âmbito Geográfico,Valor
6502,2023,Abrantes,Município,15.0
6503,2023,Águeda,Município,14.0
6504,2023,Aguiar da Beira,Município,2.0
6505,2023,Alandroal,Município,4.0
6506,2023,Albergaria-a-Velha,Município,9.0
...,...,...,...,...
6805,2023,Vimioso,Município,3.0
6806,2023,Vinhais,Município,3.0
6807,2023,Viseu,Município,26.0
6808,2023,Vizela,Município,4.0


In [255]:
df_edu_ensino_n_superior =  find_df_by_name(dataframes, '379-estabelecimentos-de-ensino-nao-superior-por-0')
df_edu_escola_final = trata_ensino_n_superior(df_edu_ensino_n_superior)
df_edu_escola_final

,Ano,Região,Privado,Público,Escola artística,Escola básica,Escola básica e secundária,Escola profissional,Escola secundária,Jardim-de-infância,Total Estabelecimentos
0,2023,Abrantes,12.0,38.0,0.0,26.0,8.0,2.0,0.0,14.0,50.0
1,2023,Aguiar da Beira,0.0,14.0,0.0,8.0,2.0,0.0,0.0,4.0,14.0
2,2023,Alandroal,0.0,8.0,0.0,6.0,0.0,0.0,0.0,2.0,8.0
3,2023,Albergaria-a-Velha,20.0,44.0,0.0,30.0,2.0,2.0,2.0,28.0,64.0
4,2023,Albufeira,14.0,46.0,0.0,32.0,0.0,0.0,4.0,24.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...
303,2023,Vouzela,4.0,22.0,0.0,16.0,0.0,2.0,2.0,6.0,26.0
304,2023,Águeda,32.0,36.0,0.0,26.0,2.0,0.0,4.0,36.0,68.0
305,2023,Évora,40.0,76.0,0.0,50.0,2.0,2.0,6.0,56.0,116.0
306,2023,Ílhavo,18.0,50.0,0.0,42.0,0.0,0.0,4.0,22.0,68.0


In [256]:
df_edu_ensino_superior =  find_df_by_name(dataframes, '375-estabelecimentos-de-ensino-superior-por-subsistema')
df_edu_ensino_superior = trata_ensino_superior(df_edu_ensino_superior)
df_edu_ensino_superior 

Tipo de Escola,Ano,Região,Privado,Público,Total
0,2024,Abrantes,0.0,1.0,1.0
1,2024,Aguiar da Beira,0.0,0.0,0.0
2,2024,Alandroal,0.0,0.0,0.0
3,2024,Albergaria-a-Velha,0.0,0.0,0.0
4,2024,Albufeira,0.0,0.0,0.0
...,...,...,...,...,...
303,2024,Vouzela,0.0,0.0,0.0
304,2024,Águeda,0.0,1.0,1.0
305,2024,Évora,0.0,6.0,6.0
306,2024,Ílhavo,0.0,0.0,0.0


In [257]:
df_econ_depositos = find_df_by_name(dataframes, '294-depositos-de-clientes-nos-bancos-caixas-economicas-e')
df_econ_depositos = trata_econ_depositos(df_econ_depositos)
df_econ_depositos

Deposito,Ano,Região,De emigrantes,De outros clientes,Total
0,2023,Abrantes,10206.0,512889.0,523095.0
1,2023,Aguiar da Beira,19138.0,162557.0,181695.0
2,2023,Alandroal,0.0,0.0,0.0
3,2023,Albergaria-a-Velha,8636.0,319967.0,328603.0
4,2023,Albufeira,21619.0,1300842.0,1322461.0
...,...,...,...,...,...
303,2023,Vouzela,7403.0,114034.0,121437.0
304,2023,Águeda,26500.0,972942.0,999442.0
305,2023,Évora,6238.0,1616903.0,1623141.0
306,2023,Ílhavo,13573.0,479636.0,493209.0


In [258]:
df_econ_estabelecimentos_bancos = find_df_by_name(dataframes, '295-estabelecimentos-de-bancos-caixas-economicas-e-caixas')
df_econ_estabelecimentos_bancos = trata_econ_bancos(df_econ_estabelecimentos_bancos)
df_econ_estabelecimentos_bancos

,Ano,Região,Âmbito Geográfico,Valor
3400,2023,Abrantes,Município,7.0
3401,2023,Águeda,Município,10.0
3402,2023,Aguiar da Beira,Município,2.0
3403,2023,Alandroal,Município,1.0
3404,2023,Albergaria-a-Velha,Município,8.0
...,...,...,...,...
3703,2023,Vimioso,Município,2.0
3704,2023,Vinhais,Município,1.0
3705,2023,Viseu,Município,33.0
3706,2023,Vizela,Município,7.0


In [259]:
df_desemprego = find_df_by_name(dataframes, '439-desemprego-registado-nos-centros-de-emprego-por-grup')
df_desemprego = trata_df_desemprego(df_desemprego)
df_desemprego

Faixa Etária,Ano,Região,25-34,35-44,45-54,55 ou mais anos,Menos de 25 anos,Total
0,2023,Abrantes,214.2,209.2,233.0,326.5,156.8,1139.6
1,2023,Aguiar da Beira,14.0,19.1,14.5,30.2,10.7,88.4
2,2023,Alandroal,9.6,16.5,22.7,24.8,10.0,83.5
3,2023,Albergaria-a-Velha,97.0,113.9,112.3,122.8,60.0,505.9
4,2023,Albufeira,674.4,532.8,416.9,430.3,224.1,2278.5
...,...,...,...,...,...,...,...,...
303,2023,Vouzela,36.5,30.2,39.0,73.8,36.1,215.5
304,2023,Águeda,225.2,221.1,194.3,238.3,139.7,1018.6
305,2023,Évora,301.8,263.8,241.6,274.8,182.3,1264.3
306,2023,Ílhavo,199.7,185.4,169.3,236.3,81.3,872.1


In [260]:
df_polacao_empregada = find_df_by_name(dataframes, '445-populacao-empregada-por-conta-de-outrem-por-nivel-de')
df_polacao_empregada = trata_populacao_empregada_escolaridade(df_polacao_empregada)
df_polacao_empregada


Escolaridade,Ano,Região,01. Inferior ao 1.º ciclo,02. Básico - 1º Ciclo,03. Básico - 2º Ciclo,04. Básico - 3º Ciclo,05. Secundário,06. Curso técnico superior profissional,07. Bacharelato,08. Licenciatura,09. Mestrado,10. Doutoramento,Total
0,2023,Abrantes,0.0,398.0,480.0,1556.0,2229.0,4.0,91.0,1039.0,130.0,0.0,5957.0
1,2023,Aguiar da Beira,0.0,121.0,143.0,282.0,292.0,0.0,5.0,102.0,13.0,0.0,965.0
2,2023,Alandroal,0.0,100.0,94.0,181.0,145.0,0.0,3.0,59.0,7.0,0.0,593.0
3,2023,Albergaria-a-Velha,24.0,386.0,965.0,1846.0,2346.0,6.0,123.0,880.0,201.0,4.0,6793.0
4,2023,Albufeira,0.0,1088.0,1683.0,5940.0,7840.0,20.0,205.0,1814.0,225.0,0.0,19102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,2023,Vouzela,0.0,108.0,259.0,469.0,641.0,0.0,13.0,235.0,42.0,0.0,1775.0
304,2023,Águeda,41.0,978.0,1904.0,3701.0,4992.0,18.0,198.0,1937.0,430.0,20.0,14225.0
305,2023,Évora,32.0,639.0,1080.0,3406.0,6354.0,13.0,142.0,2959.0,552.0,34.0,15243.0
306,2023,Ílhavo,29.0,408.0,822.0,1790.0,2748.0,4.0,108.0,1421.0,416.0,18.0,7775.0


In [261]:
df_polacao_empregada_ramo = find_df_by_name(dataframes, '845-Pessoal-ao-servico-nas-empresas-por-ramo-de-atividade')
df_polacao_empregada_ramo = trata_populacao_empregada_ramo(df_polacao_empregada_ramo)
df_polacao_empregada_ramo


Ramo de Atividade,Ano,Região,"A. Agricultura, produção animal, caça, floresta e pesca","A.01. Agricultura, produção animal, caça e atividades dos serviços relacionados",A.02. Silvicultura e exploração florestal,A.03. Pesca e aquicultura,B. Indústrias extrativas,C. Indústrias transformadoras,"D. Eletricidade, gás, vapor, água quente e fria e ar frio","E. Captação, tratamento e distribuição de água; saneamento, gestão de resíduos e despoluição",...,"M. Atividades de consultoria, científicas, técnicas e similares",M.69. Atividades jurídicas e de contabilidade,M.70. Atividades das sedes sociais e de consultoria para a gestão,"M.71. Atividades de arquitetura, de engenharia e técnicas afins; atividades de ensaios e de análises técnicas",N. Atividades administrativas e dos serviços de apoio,P. Educação,Q. Atividades de saúde humana e apoio social,"R. Atividades de teatro, de música, de dança e outras atividades artísticas e literárias",S. Outras atividades de serviços,Total
0,2023,Abrantes,487.0,241.0,231.0,15.0,0.0,2076.0,113.0,197.0,...,584.0,161.0,39.0,164.0,443.0,189.0,628.0,126.0,307.0,8514.0
1,2023,Aguiar da Beira,285.0,263.0,22.0,0.0,30.0,169.0,4.0,0.0,...,73.0,32.0,0.0,24.0,32.0,16.0,85.0,8.0,33.0,1717.0
2,2023,Alandroal,473.0,464.0,9.0,0.0,8.0,118.0,0.0,0.0,...,31.0,10.0,0.0,13.0,102.0,10.0,24.0,21.0,16.0,1192.0
3,2023,Albergaria-a-Velha,416.0,226.0,155.0,35.0,0.0,4425.0,12.0,47.0,...,440.0,165.0,71.0,115.0,520.0,0.0,270.0,86.0,164.0,10547.0
4,2023,Albufeira,593.0,503.0,13.0,77.0,0.0,508.0,13.0,0.0,...,1181.0,496.0,257.0,165.0,2783.0,307.0,591.0,583.0,768.0,28110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,2023,Vouzela,434.0,327.0,107.0,0.0,0.0,495.0,11.0,5.0,...,143.0,42.0,12.0,60.0,124.0,82.0,80.0,24.0,48.0,2432.0
304,2023,Águeda,418.0,291.0,124.0,3.0,62.0,11455.0,63.0,36.0,...,903.0,401.0,143.0,234.0,1194.0,257.0,497.0,228.0,391.0,22562.0
305,2023,Évora,2019.0,1875.0,0.0,0.0,5.0,4654.0,13.0,257.0,...,1400.0,443.0,213.0,277.0,2406.0,469.0,1632.0,413.0,531.0,22464.0
306,2023,Ílhavo,912.0,0.0,0.0,795.0,57.0,5713.0,24.0,7.0,...,914.0,261.0,122.0,263.0,1106.0,354.0,427.0,161.0,371.0,15737.0


In [262]:
df_populacao_estrangeira = find_df_by_name(dataframes, '354-populacao-estrangeira-com-estatuto-legal-de-residente')
df_populacao_estrangeira = trata_populacao_estrangeira(df_populacao_estrangeira)
df_populacao_estrangeira

Nacionalidade,Ano,Região,1. África,1.1. Angola,1.2.Cabo-Verde,1.3. Guiné-Bissau,1.4. Moçambique,1.5. São Tomé e Príncipe,2. América,2.1. Brasil,...,3.3. Nepal,4. Europa,4.1. Espanha,4.2. França,4.3. Itália,4.4. República da Moldova,4.5. Reino Unido,4.6. Roménia,4.7. Ucrânia,Total
0,2023,Abrantes,0.0,191.0,22.0,23.0,0.0,3.0,0.0,475.0,...,0.0,0.0,0.0,0.0,0.0,4.0,40.0,76.0,39.0,1169.0
1,2023,Aguiar da Beira,0.0,2.0,2.0,0.0,0.0,0.0,0.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,7.0,63.0
2,2023,Alandroal,0.0,2.0,0.0,1.0,0.0,1.0,0.0,76.0,...,0.0,0.0,0.0,0.0,0.0,1.0,20.0,6.0,7.0,189.0
3,2023,Albergaria-a-Velha,0.0,34.0,4.0,13.0,0.0,10.0,0.0,587.0,...,0.0,0.0,0.0,0.0,0.0,6.0,13.0,2.0,79.0,1181.0
4,2023,Albufeira,0.0,115.0,314.0,557.0,0.0,31.0,0.0,5039.0,...,0.0,0.0,0.0,0.0,0.0,223.0,2742.0,497.0,968.0,18970.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,2023,Vouzela,0.0,6.0,0.0,0.0,0.0,21.0,0.0,138.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,10.0,260.0
304,2023,Águeda,0.0,64.0,43.0,39.0,0.0,44.0,0.0,1933.0,...,0.0,0.0,0.0,0.0,0.0,3.0,19.0,16.0,128.0,3260.0
305,2023,Évora,0.0,177.0,283.0,59.0,0.0,19.0,0.0,1826.0,...,0.0,0.0,0.0,0.0,0.0,20.0,42.0,48.0,82.0,3684.0
306,2023,Ílhavo,0.0,67.0,18.0,122.0,0.0,17.0,0.0,1392.0,...,0.0,0.0,0.0,0.0,0.0,4.0,14.0,38.0,67.0,2630.0


In [263]:
df_moradia_edificios = find_df_by_name(dataframes, '978-edificios-concluidos-por-tipo-de-obra-e-fim-que-se')
df_moradia_edificios = trata_moradia_edificios(df_moradia_edificios)
df_moradia_edificios

,Ano,Região,Habitação familiar,Outros,"Ampliações, alterações e reconstruções",Construções novas,Total_y
0,2023,Abrantes,30.0,68.0,22.0,76.0,98.0
1,2023,Aguiar da Beira,30.0,6.0,8.0,28.0,36.0
2,2023,Alandroal,22.0,8.0,14.0,16.0,30.0
3,2023,Albergaria-a-Velha,40.0,52.0,4.0,88.0,92.0
4,2023,Albufeira,126.0,10.0,72.0,64.0,136.0
...,...,...,...,...,...,...,...
303,2023,Vouzela,44.0,24.0,0.0,68.0,68.0
304,2023,Águeda,86.0,40.0,16.0,110.0,126.0
305,2023,Évora,90.0,66.0,36.0,120.0,156.0
306,2023,Ílhavo,162.0,102.0,10.0,254.0,264.0


In [264]:
df_construcoes_novas = find_df_by_name(dataframes, '980-fogos-concluidos-em-construcoes-novas-para-habitacao')
df_construcoes_novas = trata_construcoes_novas(df_construcoes_novas)
df_construcoes_novas

Qtd. Quartos,Ano,Região,T0 ou T1,T2,T3,T4 ou mais,Total
0,2023,Abrantes,0.0,1.0,9.0,2.0,12.0
1,2023,Aguiar da Beira,0.0,2.0,6.0,3.0,11.0
2,2023,Alandroal,0.0,4.0,2.0,0.0,6.0
3,2023,Albergaria-a-Velha,0.0,3.0,16.0,1.0,20.0
4,2023,Albufeira,54.0,39.0,46.0,10.0,149.0
...,...,...,...,...,...,...,...
303,2023,Vouzela,0.0,8.0,11.0,4.0,23.0
304,2023,Águeda,2.0,9.0,31.0,2.0,44.0
305,2023,Évora,1.0,7.0,23.0,30.0,61.0
306,2023,Ílhavo,22.0,59.0,81.0,18.0,180.0


In [265]:
df_sal_ganho_medio_mensal= find_df_by_name(dataframes, '581-ganho-medio-mensal')
df_sal_ganho_medio_mensal = trata_sal_ganho_medio_mensal(df_sal_ganho_medio_mensal)
df_sal_ganho_medio_mensal


,Ano,Região,Âmbito Geográfico,Valor
5872,2023,Abrantes,Município,1289.0
5873,2023,Águeda,Município,1280.2
5874,2023,Aguiar da Beira,Município,1016.9
5875,2023,Alandroal,Município,1171.4
5876,2023,Albergaria-a-Velha,Município,1349.1
...,...,...,...,...
6175,2023,Vimioso,Município,1006.4
6176,2023,Vinhais,Município,998.2
6177,2023,Viseu,Município,1255.0
6178,2023,Vizela,Município,1104.9


In [266]:
df_sal_ganho_medio_mensal_escol= find_df_by_name(dataframes, '582-ganho-medio-mensal-por-nivel-de-escolaridade')
df_sal_ganho_medio_mensal_escol = trata_sal_ganho_medio_mensal_escol(df_sal_ganho_medio_mensal_escol)
df_sal_ganho_medio_mensal_escol

Nível de Escolaridade,Ano,Região,01. Inferior ao 1.º ciclo,02. Básico - 1º Ciclo,03. Básico - 2º Ciclo,04. Básico - 3º Ciclo,05. Secundário,06. Curso técnico superior profissional,07. Bacharelato,08. Licenciatura,09. Mestrado,10. Doutoramento,Total
0,2023,Abrantes,959.6,1106.7,1079.8,1107.7,1176.2,1722.5,2018.5,1771.7,2259.5,1932.8,1289.0
1,2023,Aguiar da Beira,0.0,1024.3,930.2,964.2,980.5,0.0,1921.1,1240.8,1549.7,0.0,1016.9
2,2023,Alandroal,0.0,1032.1,1143.6,1037.2,1170.8,0.0,1069.8,1781.2,1937.2,0.0,1171.4
3,2023,Albergaria-a-Velha,983.3,1125.4,1215.9,1236.3,1273.5,2078.0,1900.0,1825.7,1893.8,2553.2,1349.1
4,2023,Albufeira,1010.3,1105.0,1098.0,1105.0,1217.7,1738.1,1660.7,1625.3,2063.2,0.0,1216.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,2023,Vouzela,891.8,1019.7,966.0,1028.1,1082.9,0.0,1357.6,1562.9,1611.7,0.0,1127.6
304,2023,Águeda,961.9,1085.3,1146.8,1151.2,1199.1,1528.0,1689.4,1774.5,1905.3,2767.3,1280.2
305,2023,Évora,1043.2,1061.0,1096.5,1160.1,1236.2,1423.5,1770.6,1856.2,2287.9,2905.6,1369.7
306,2023,Ílhavo,907.1,1107.3,1206.3,1196.5,1248.0,1272.9,1955.6,1913.5,2048.4,3363.5,1403.3


In [ ]:
df_econ_poder_compra = trata_econ_poder_compra()
df_econ_poder_compra


In [ ]:
# liste as cidades com amior poder de compra usando df_econ_poder_compra
# imprima na tela

df_econ_poder_compra = df_econ_poder_compra.sort_values(by='Valor', ascending=False).head(100)
print("Cidades com maior poder de compra:")
print(df_econ_poder_compra[['Região', 'Valor']])


Cidades com maior poder de compra:
                 Região  Valor
223              Lisboa  9.748
228              Sintra  3.636
38                Porto  3.343
47    Vila Nova de Gaia  2.964
227              Oeiras  2.729
..                  ...    ...
103           Estarreja  0.219
101              Anadia  0.219
300               Lagoa  0.211
276          Portalegre  0.210
100  Albergaria-a-Velha  0.207

[100 rows x 2 columns]
